<a href="https://colab.research.google.com/github/ningjingzhiyuan507/AnalysisTool/blob/master/d2l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 配置GPU版本pytorch

In [ ]:
!pip list

In [3]:
# 配置pytorch环境
!pip install torch==1.8.1
!pip install torchvision==0.9.1

     |████████████████████████████████| 804.1 MB 2.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
     |████████████████████████████████| 17.4 MB 602 kB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111


In [6]:
# 测试GPU
import torch
from torch import nn

In [9]:
torch.device('cuda')

device(type='cuda')

In [10]:
torch.cuda.device_count()

0

In [12]:
X = torch.rand(2, 3, device=try_gpu(1))

NameError: ignored